In [1]:
import pandas as pd
import numpy as np

In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../data/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../data/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../data/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('../data/uid_test_a.txt',index=None)

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

##  baseline 

### 通话记录

In [6]:
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()

voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)

voice_opp_len['call_to_phone'] = voice_opp_len['voice_opp_len_11']+ voice_opp_len['voice_opp_len_12']

voice_call_to_phone = voice_opp_len[['uid', 'call_to_phone']]

# voice['call_duration'] = voice['end_time'].astype(float) - voice['start_time'].astype(float)
# voice_avg_call_time = voice.groupby(['uid'])['call_duration'].agg({'call_duration_mean':'mean'}).reset_index()

# todo： 通话开始时间的平均值，方差

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


## 短信记录

In [7]:
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()
# sms_opp_num['sms_opp_num_rate'] = sms_opp_num['sms_opp_num_count'] / sms_opp_num['sms_opp_num_unique_count']

sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)

# sms_opp_len['sms_to_phone'] = sms_opp_len['sms_opp_len_11']+ sms_opp_len['sms_opp_len_12']
# sms_send_to_phone = sms_opp_len[['uid', 'sms_to_phone']]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


### 网站/APP记录

In [8]:
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()
# visit_dura['avg_dura'] = visit_dura['wa_visit_dura_sum'] / wa_name['wa_name_count']
# visit_dura['avg_unique_dura'] = visit_dura['wa_visit_dura_sum'] / wa_name['wa_name_unique_count']


up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()

#wa_type = wa.groupby(['uid'])['wa_type'].agg({'sum_count':'count', 'app_count':'sum'}).add_prefix('wa_type_').reset_index()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [9]:
# 我加入的
a = sms.groupby(['uid', 'opp_num'])['opp_num'].agg({'count':'count'}).add_prefix('voice_opp_num_').reset_index()
a = a.groupby(['uid'])['voice_opp_num_count'].agg(['std','max','min','median','mean','sum']).fillna(0).add_prefix('voice_opp_num_').reset_index()

b = sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_head_').reset_index().fillna(0)

# feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow, down_flow, voice_call_to_phone, voice_avg_call_time, sms_send_to_phone, wa_type]
feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
down_flow, voice_call_to_phone, a, b]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


In [10]:
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')
train_feature = train_feature.fillna(0)

In [11]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')
test_feature = test_feature.fillna(0)

In [12]:
train_feature.to_csv('../data/train_featureV2.csv',index=None)
test_feature.to_csv('../data/test_featureV2.csv',index=None)

In [13]:
sms

,uid,opp_num,opp_head,opp_len,start_time,in_out
0,u4003,B378E065731B897E7295926B27CBA0D5,186,11,20174042,1
1,u4003,B378E065731B897E7295926B27CBA0D5,186,11,20174130,1
2,u4003,1B15607F3E6D167B44D46046D5993D87,189,11,20015746,0
3,u4003,1B15607F3E6D167B44D46046D5993D87,189,11,20015640,0
4,u4003,B378E065731B897E7295926B27CBA0D5,186,11,20045300,1
5,u4003,B378E065731B897E7295926B27CBA0D5,186,11,20045253,1
6,u2354,FFCB99E153F74A7A6BB2481E9EAA666D,132,11,20104305,1
7,u2354,FFCB99E153F74A7A6BB2481E9EAA666D,132,11,20104305,1
8,u2354,FFCB99E153F74A7A6BB2481E9EAA666D,132,11,20170619,1
9,u2354,7FEFA80914C7C9336EEC31BAEC427E14,106,9,20090930,1


In [14]:
a.dtypes

uid                      object
voice_opp_num_std       float64
voice_opp_num_max         int64
voice_opp_num_min         int64
voice_opp_num_median    float64
voice_opp_num_mean      float64
voice_opp_num_sum         int64
dtype: object

In [15]:
b = sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_head_').reset_index()
b

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,uid,sms_head_count,sms_head_unique_count
0,u0001,95,6
1,u0002,1,1
2,u0003,28,2
3,u0004,124,8
4,u0005,48,4
5,u0006,30,3
6,u0007,42,2
7,u0008,379,7
8,u0009,15,2
9,u0010,34,2
